In [31]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import src.graph_ops as go
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from torch.utils import data
from sklearn.preprocessing import OneHotEncoder

In [32]:
graphs = np.load("data/graph_10node7500.npy")
# options: graph_10+20k, graph_10+10k
labels = np.load("data/labels_10node7500.npy")
# options: labels_10+20k, labels_10+10k

In [ ]:
X = graphs
X = X[:,None]

In [34]:
y = pd.DataFrame(np.zeros((labels.shape[0], 10)))
y = y.add(pd.get_dummies(labels[:,0]), fill_value=0)
y = y.add(pd.get_dummies(labels[:,1]), fill_value=0)
y = y.values.astype(int)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [46]:
def cross_entropy(input, target, size_average=True):
    return torch.mean(-target * torch.log(input))


class CNN(nn.Module): 
    
    #constructor
    #take in X as a parameter
    def __init__(self, X, y, weight_decay = 0.0005):
        super(CNN, self).__init__()
        
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        #self.device = 'cpu'
        
        y_dim = y.shape[-1]
        
        self.conv1 = torch.nn.Conv2d(1, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.fc1 = torch.nn.Linear(18 * 5 * 5, 100)
        
        self.fc2 = torch.nn.Linear(100, y_dim)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001, weight_decay = weight_decay)
        self.to(self.device)
        
    # 1. input X
    def forward(self, X):
        X = F.relu(self.conv1(X))
        
        X = self.pool(X)
    
        X = X.view(-1, int(18 * 5 * 5))
        
        X = self.relu(self.fc1(X))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        X = self.fc2(X)
        X = self.sigmoid(X)
        X = 2*X/(X.sum(1)[:,None])
        return X
    
    def loss(self, pred, true):
        #PyTorch's own cross entropy loss function.
        score = cross_entropy
        l = None
        for i in range(true.shape[1]):
            if l is None:
                l = score(pred[:,i], true[:,i])
            else:
                l += score(pred[:,i], true[:,i])
        return l
    

    def fit(self, X, y, early_stopping = True, patience = 50):
        
        if isinstance(X, pd.DataFrame):
            X_arr = X.values
            X_tens = Variable(torch.Tensor(X_arr).float())
        else:
            X_tens = Variable(torch.Tensor(X).float())
        if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
            y_arr = y.values
            if len(y_arr.shape) == 1:
                y_arr = y_arr[:,None]
            y_tens = Variable(torch.Tensor(y_arr).float())
        else:
            if len(y.shape) == 1:
                y = y[:,None]
            y_tens = Variable(torch.Tensor(y).float())
        
        X_tens = X_tens.to(self.device)
        y_tens = y_tens.to(self.device)
        
        if early_stopping == True:
            
            inds = np.arange(len(X_tens))
            num_train = int(len(X_tens) * .9)
            train_inds = np.random.choice(inds, num_train, replace=False)
            val_inds = np.setdiff1d(inds, train_inds)
            X_train = X_tens[train_inds]
            y_train = y_tens[train_inds]
            X_val = X_tens[val_inds]
            y_val = y_tens[val_inds]
            
        for epoch in range(1000):
            self.optimizer.zero_grad()
            if early_stopping==True:
                out_train = self.forward(X_train)
                loss = self.loss(out_train, y_train)
            if early_stopping==False:
                out_train = self.forward(X_tens)
                loss = self.loss(out_train, y_tens)
                print(loss)
                loss.backward()
                self.optimizer.step()
                continue
                
            out_val = self.forward(X_val)
            loss_val = self.loss(out_val, y_val)
            print(loss_val)


            if epoch == 0:
                min_loss = loss_val
                best_weights = self.state_dict()
                counter = 0
            else:
                if loss_val < min_loss:
                    min_loss = loss_val
                    best_weights = self.state_dict()
                    counter = 0
                else:
                    counter += 1
                    if counter == patience:
                        self.load_state_dict(best_weights)
                        return
                
            loss.backward()
            self.optimizer.step()
    
    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X_arr = X.values
            X_tens = Variable(torch.Tensor(X_arr).float())
        else:
            X_tens = Variable(torch.Tensor(X).float())
            
        X_tens = X_tens.to(self.device)
        
        predict_out = self.forward(X_tens)
        temp =  predict_out.cpu().detach().numpy()
        temp[temp > 1-1e-6] = 1-1e-6
        temp[temp < 1e-6]=1e-6
        return temp
    
    def predict(self, X):
        proba = self.predict_proba(X).argsort(1)[:,-2:]
        proba.sort(1)
        return proba
    
    def score(self, X, y):
        y_pred = self.predict(X)
        y_true = y.argsort(1)[:,-2:]
        y_true.sort(1)
        return (y_pred == y_true).all(1).mean().item()
    
    def top_k_acc(self, X,y, k = 5):
        """Get top k accuracy for prediction. I.e., were the 2 correct nodes 
        in the top k returned. Note, k=2 is the score function."""
        y_pred = self.predict_proba(X)
        y_true = y.argsort(1)[:,-2:]  
        y_true.sort(1)
        eqs_list = []
        top_k = y_pred.argsort(1)[:,-k:]
        for i in range(y_true.shape[0]):
            row = y_true[i]
            eqs_arr = []
            for ele in row:
                eqs_arr.append(np.any(ele == top_k[i]))
            eqs_arr = np.array(eqs_arr)
            eq = eqs_arr.all()
            eqs_list.append(eq)
        eqs = np.array(eqs_list)

        return eqs.mean()

In [47]:
cnn = CNN(X, y)

cnn.fit(X_train,y_train)

tensor(3.1636, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0386, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.9353, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.8451, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7642, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6917, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6273, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5707, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5212, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4785, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4425, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4126, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3882, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3682, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3511, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3358, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3221, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3105, device='cuda:0', grad_fn=<AddBack

tensor(1.0538, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0491, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0428, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0373, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0327, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0272, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0222, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0175, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0117, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0086, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0022, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0001, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9932, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9924, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9853, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9845, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9762, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9738, device='cuda:0', grad_fn=<AddBack

tensor(0.5999, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5992, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5973, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5967, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5951, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5940, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5931, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5915, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5911, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5894, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5891, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5876, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5872, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5856, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5853, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5837, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5831, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5817, device='cuda:0', grad_fn=<AddBack

tensor(0.5331, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5357, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5326, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5353, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5322, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5331, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5326, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5319, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5338, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5313, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5335, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5311, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5323, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5320, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5312, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5324, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5311, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5323, device='cuda:0', grad_fn=<AddBack

In [48]:
cnn.score(X_test,y_test)

0.9139670738183749

In [51]:
cnn_acc = [cnn.top_k_acc(X_test, y_test, k = i) for i in range(2,11)]

In [52]:
cnn_acc

[0.9139670738183749,
 0.9373340414232607,
 0.9564524694636218,
 0.966542750929368,
 0.9750398300584174,
 0.9835369091874668,
 0.9904407859798194,
 0.9941582580987786,
 1.0]